In [1]:
import numpy as np
import struct


class MnistData:
    def __init__(self, train_image_path, train_label_path,
                 test_image_path, test_label_path):
        # 训练集和验证集的文件路径
        self.train_image_path = train_image_path
        self.train_label_path = train_label_path
        self.test_image_path = test_image_path
        self.test_label_path = test_label_path

        # 获取训练集和验证集数据
        # get_data()方法，参数为0获取训练集数据，参数为1获取验证集
        self.train_images, self.train_labels = self.get_data(0)
        self.test_images, self.test_labels = self.get_data(1)

    def get_data(self, data_type):
        if data_type == 0:  # 获取训练集数据
            image_path = self.train_image_path
            label_path = self.train_label_path
        else:  # 获取验证集数据
            image_path = self.test_image_path
            label_path = self.test_label_path

        with open(image_path, 'rb') as file1:
            image_file = file1.read()
        with open(label_path, 'rb') as file2:
            label_file = file2.read()

        label_index = 0
        image_index = 0
        labels = []
        images = []

        # 读取训练集图像数据文件的文件信息
        magic, num_of_datasets, rows, columns = struct.unpack_from('>IIII',
                                                                   image_file, image_index)
        image_index += struct.calcsize('>IIII')

        for i in range(num_of_datasets):
            # 读取784个unsigned byte，即一副图像的所有像素值
            temp = struct.unpack_from('>784B', image_file, image_index)
            # 将读取的像素数据转换成28*28的矩阵
            temp = np.reshape(temp, (28, 28))
            # 归一化处理
            temp = temp / 255
            images.append(temp)
            image_index += struct.calcsize('>784B')  # 每次增加784B

        # 跳过描述信息
        label_index += struct.calcsize('>II')
        labels = struct.unpack_from('>' + str(num_of_datasets) + 'B', label_file, label_index)

        # one-hot
        labels = np.eye(10)[np.array(labels)]

        return np.array(images), labels

In [2]:
train_image_path = './data/train-images-idx3-ubyte'
train_label_path = './data/train-labels-idx1-ubyte'
test_image_path = './data/t10k-images-idx3-ubyte'
test_label_path = './data/t10k-labels-idx1-ubyte'

data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path)

print(type(data.train_images))
print(type(data.train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [3]:
print(type(data.train_images))
print(type(data.train_labels))

print(type(data.train_images[0]))
print(type(data.train_labels[0]))

print(data.train_images[0])
print(data.train_labels[0])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.      

In [7]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [11]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.LSTM(128, input_shape=(784, 1), activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

W0612 05:24:07.434529 139725216646912 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f13b8412438>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


ValueError: Input 0 of layer unified_lstm_6 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 784]

In [8]:
model.fit(data.train_images, data.train_labels, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 6s 97us/sample - loss: 0.2224 - accuracy: 0.9337
Epoch 2/10
60000/60000 [==============================] - 6s 96us/sample - loss: 0.0961 - accuracy: 0.9705
Epoch 3/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0691 - accuracy: 0.9776
Epoch 4/10
60000/60000 [==============================] - 5s 92us/sample - loss: 0.0523 - accuracy: 0.9836
Epoch 5/10
60000/60000 [==============================] - 5s 91us/sample - loss: 0.0416 - accuracy: 0.9865
Epoch 6/10
60000/60000 [==============================] - 5s 91us/sample - loss: 0.0369 - accuracy: 0.9878
Epoch 7/10
60000/60000 [==============================] - 5s 91us/sample - loss: 0.0308 - accuracy: 0.9895
Epoch 8/10
60000/60000 [==============================] - 5s 92us/sample - loss: 0.0257 - accuracy: 0.9911
Epoch 9/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0255 - accuracy: 0.9915
Epoch 10/10
60000/60000 [============

In [10]:
model.evaluate(data.test_images, data.test_labels)

10000/10000 [==============================] - 1s 66us/sample - loss: 0.0731 - accuracy: 0.9817


[0.07307186364395202, 0.9817]

In [5]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data.train_images, data.train_labels, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 6s 97us/sample - loss: 0.2205 - accuracy: 0.9349
Epoch 2/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0982 - accuracy: 0.9704
Epoch 3/10
60000/60000 [==============================] - 6s 92us/sample - loss: 0.0694 - accuracy: 0.9782
Epoch 4/10
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0531 - accuracy: 0.9827
Epoch 5/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0437 - accuracy: 0.9855
Epoch 6/10
60000/60000 [==============================] - 5s 89us/sample - loss: 0.0362 - accuracy: 0.9883
Epoch 7/10
60000/60000 [==============================] - 5s 91us/sample - loss: 0.0291 - accuracy: 0.9897
Epoch 8/10
60000/60000 [==============================] - 5s 90us/sample - loss: 0.0273 - accuracy: 0.9907
Epoch 9/10
60000/60000 [==============================] - 5s 90us/sample - loss: 0.0241 - accuracy: 0.9919
Epoch 10/10
60000/60000 [============

In [17]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(type(x_train))
print(type(x_test))

# print(x_train[0])
print(y_train[0])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
5
